## Identifying a Proteomic Signature of Severe COVID-19
### Cohort Overview
#### C.V. Cosgriff, MD, MPH

### 0- Environment

In [1]:
import pandas as pd
import numpy as np

from tableone import TableOne

### 1- Features and Outcomes

Load MESSI proteomics dataset.

In [2]:
df = pd.read_csv('../data/MESSI-COVID-Proteomics_final-dat.csv')

Generate outcome variables on ordinal scale and binary versions.

In [3]:
# ordinal severity
nihord_d00 = pd.Categorical(df.ordinal_enrollment, ordered=True)
nihord_d07 = pd.Categorical(df.ordinal_d7, ordered=True)
nihord_d14 = pd.Categorical(df.ordinal_d14, ordered=True)
nihord_d28 = pd.Categorical(df.ordinal_d28, ordered=True)

df = df.assign(nihord_d00=nihord_d00, nihord_d07=nihord_d07,
               nihord_d14=nihord_d14, nihord_d28=nihord_d28)

# binary severity
mod_sev_d0 = df.nihord_d00 <= 3
mod_sev_d7 = df.nihord_d07 <= 3
mod_sev_d14 = df.nihord_d14 <= 3
mod_sev_d28 = df.nihord_d28 <= 3

df = df.assign(mod_sev_d0=mod_sev_d0, mod_sev_d7=mod_sev_d7,
               mod_sev_d14=mod_sev_d14, mod_sev_d28=mod_sev_d28)

# mortality
mortality_map = {'Alive' : 0, 'Dead' : 1, 'Unknown' : np.nan}
mort90_cons = df.mortality_90d.map(mortality_map)
df = df.assign(mort90_cons=mort90_cons)

Process demographic features.

In [4]:
# sex
male_sex = (df.sex == 'Male').astype(int)
df = df.assign(male_sex=male_sex)

# self-reported race
print(df.race.value_counts())

# code race as binary, use AA as reference given size
race_BAA = df.race == 'Black'
df = df.assign(race_BAA=race_BAA.astype(int))

# co-morbidity
cmdz = df.mhcad + df.mhhtn + df.mhdm + df.mhcri
mhcmdz = (cmdz > 0).astype(int)
df = df.assign(mhcmdz=mhcmdz)

Black             112
White              46
Asian               7
Other               1
Pacific Island      1
Name: race, dtype: int64


### 2- Table One

In [5]:
# generate a list of features for table 1 and their 'prettier' labels
columns = ['age', 'sex', 'race', 'ethnicity', 'mhcmdz', 'early_steroids', 'ordinal_enrollment', 'apacheIII', 'vasoactives_yn', 'mortality_90d', 'mod_sev_d0']
labels = {'age_calculated' : 'Age', 'sex' : 'Male Sex', 'race' : 'Race',
          'ethnicity' : 'Hispanic', 'ordinal_enrollment' : 'WHO Ordinal Enrollment',
         'mhcmdz' : 'Cardiometabolic Co-morbdidity', 'apacheIII' : 'APACHE III',
         'early_steroids' : 'Steroids Before First Blood Draw', 'mortality_90d' : '90-day Mortality',
         'vasoactives_yn' : 'Vasopressors'}

# demarcate the categorical variables versus numerical
categorical = ['sex', 'race', 'ethnicity', 'mhcmdz', 'early_steroids', 'vasoactives_yn', 'ordinal_enrollment', 'mortality_90d']

# delineate the grouping variable
groupby = ['mod_sev_d0']

remap = {True : 'Severe', False : 'Non-severe'}

df_temp = df.copy()
df_temp.mod_sev_d0 = df_temp.mod_sev_d0.map(remap)

# generate and print table
cohort_overview = TableOne(df_temp.loc[:, columns], columns=columns, categorical=categorical,
                   rename=labels, groupby=groupby, pval=False)
display(cohort_overview)

/Users/cvc/opt/anaconda3/lib/python3.9/site-packages/tableone/tableone.py:991: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['percent'] = df['freq'].div(df.freq.sum(level=0),


Grouped by mod_sev_d0                                       
                                                                     Missing      Overall   Non-severe       Severe
n                                                                                     167           76           91
age, mean (SD)                                                             0  58.9 (14.6)  57.1 (15.6)  60.4 (13.6)
Male Sex, n (%)                         Female                             0    76 (45.5)    36 (47.4)    40 (44.0)
                                        Male                                    91 (54.5)    40 (52.6)    51 (56.0)
Race, n (%)                             Asian                              0      7 (4.2)      2 (2.6)      5 (5.5)
                                        Black                                  112 (67.1)    56 (73.7)    56 (61.5)
                                        Other                                     1 (0.6)                   1 (1.1)
                                        Pacific Island                            1 (0.6)                   1 (1.1)
                                        White                                   46 (27.5)    18 (23.7)    28 (30.8)
Hispanic, n (%)                         Hispanic                           0     12 (7.2)      5 (6.6)      7 (7.7)
                                        Not hispanic                           155 (92.8)    71 (93.4)    84 (92.3)
Cardiometabolic Co-morbdidity, n (%)    0                                  0    41 (24.6)    17 (22.4)    24 (26.4)
                                        1                                      126 (75.4)    59 (77.6)    67 (73.6)
Steroids Before First Blood Draw, n (%) 0.0                                2   105 (63.6)    65 (87.8)    40 (44.0)
                                        1.0                                     60 (36.4)     9 (12.2)    51 (56.0)
WHO Ordinal Enrollment, n (%)           2.0                                0    49 (29.3)                 49 (53.8)
                                        3.0                                     42 (25.1)                 42 (46.2)
                                        4.0                                     43 (25.7)    43 (56.6)             
                                        5.0                                     33 (19.8)    33 (43.4)             
APACHE III, mean (SD)                                                      0  50.6 (30.8)  33.6 (17.5)  64.8 (32.4)
Vasopressors, n (%)                     0.0                               44    73 (59.3)    47 (94.0)    26 (35.6)
                                        1.0                                     50 (40.7)      3 (6.0)    47 (64.4)
90-day Mortality, n (%)                 Alive                              0   133 (79.6)    69 (90.8)    64 (70.3)
                                        Dead                                    31 (18.6)      5 (6.6)    26 (28.6)
                                        Unknown                                   3 (1.8)      2 (2.6)      1 (1.1)

### 3- Output Updated Dataset for Pathway Analysis

In [6]:
df.to_csv('../data/MESSI-COVID-Proteomics_final-dat_processed.csv')